In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('spotify_millsongdata.csv')

In [ ]:
df.shape

(57650, 4)

In [ ]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df['text'][0]

"I'm dancing like a mad man  \r\nI hope she likes-a me  \r\nThe chance to be just on my own  \r\nI want to silly be  \r\n  \r\nThe club is full of people  \r\nPerhaps they think I'm mad  \r\nI close my eyes and I go home  \r\nI feel a little sad  \r\n  \r\n[Chorus]  \r\nI just can't dance  \r\nMy only chance  \r\n  \r\nI'm shy, shy, oh so shy  \r\n  \r\nThe girls don't like the way I move  \r\nI think at times I've lost it  \r\nDo I look like I can't move  \r\nOr do I move exhausted  \r\n  \r\nIf I just knew she loves me  \r\nI know I wouldn't care  \r\nBut she talks to the barman  \r\nAnd I think that's unfair  \r\n  \r\n[Chorus]  \r\n  \r\nI'm sad, sad, oh so sad  \r\n  \r\nAnd now that girl moves close to me  \r\nI join into her move  \r\nPretending I enjoy it  \r\nAnd now I lost the groove  \r\n  \r\nOne guy stands tho', between us  \r\nShows up with all his power  \r\nI now feel like an idiot  \r\nWho stands in a cold shower  \r\n  \r\n[Chorus]  \r\n  \r\nThe barman stares at me  

In [ ]:
#df = df.sample(5000)

In [ ]:
df.shape

(5000, 3)

In [ ]:
#Text Cleaning
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
nltk.download('punkt')

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
token("you are beautiful")

'you are beauti'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
matrix = tfid.fit_transform(df['text'])

In [ ]:
matrix

<5000x17516 sparse matrix of type '<class 'numpy.float64'>'
	with 278004 stored elements in Compressed Sparse Row format>

In [ ]:
similar = cosine_similarity(matrix)

In [ ]:
similar[0]

array([1.        , 0.03318646, 0.01800696, ..., 0.02409376, 0.04448143,
       0.01430884])

In [ ]:
df.head(2)

,artist,song,text
0,Yello,Rock Stop,i 'm danc like a mad man i hope she likes-a me...
1,Doors,When The Music's Over,"yeah , c'mon when the music 's over when the m..."


In [ ]:
df[df['song']=="Rock Stop"].index[0]

0

In [ ]:
#Recommender Function
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song


In [ ]:
recommender("Rock Stop")

["I'm A Shy Guy", 'Above Ground', 'How Will I Know', 'Dance']

In [ ]:
import pickle
pickle.dump(similar, open('similarity.pkl', 'wb'))
pickle.dump(df, open('df.pkl', 'wb'))